In [14]:
import pandas as pd
import time

from utils.spotify import SpotifyWrapper
from utils.postgres import Postgres

db = Postgres()
spot = SpotifyWrapper()

 # Read from album table in DB
song_uris = pd.DataFrame(db.fetch_data(f"""
        SELECT song_uri, song_name FROM songs;
        """), columns = ["song_uri", "song_name"]
)

def get_song_metrics(uris):
    """
    Extracts audio features for a list of songs and loads to DB
    
    Parameters:
        uris (pd DF): 2 columns: song_uri, song_name
    
    Returns:
        None, loads DB
    """
    print(type(uris))
    #print(uris)
    total_songs = len(uris)
    print(f"Fetching audio features for {total_songs} songs.")

    columns = ["song_uri", "song_name", "duration_ms", "popularity", "danceability", "energy", "loudness", "valence", \
        "tempo", "instrumentalness","speechiness"]
    for count in range(0,25):#total_songs):
        if(i%50 == 0):
            print("Song # "+str(count)+" of "+ str(total_songs ))
    
        song_uri = uris.iloc[count]["song_uri"]
        song_name = uris.iloc[count]["song_name"]

        audio_features = spot.getAudioFeatures(song_uri)
        track_data = spot.getTrack(song_uri)

        duration = track_data['duration_ms'] * 0.001
        popularity = track_data['popularity']

        danceability = audio_features[0]['danceability']
        energy = audio_features[0]['energy']
        loudness = audio_features[0]['loudness']
        valence = audio_features[0]['valence']
        tempo = audio_features[0]['tempo']
        instrumentalness = audio_features[0]['instrumentalness']
        speechiness = audio_features[0]['speechiness']

        #
        if (not audio_features or not track_data):
            print(f"{song_name} not found. skipping.")
        else:
            db.execute_query(f"""
                INSERT INTO metrics (song_uri, song_name, duration, popularity, danceability, energy, loudness, valence, tempo, instrumentalness, speechiness)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                    ON CONFLICT (song_uri)
                    DO NOTHING
                """, (song_uri, song_name, duration, popularity, danceability, energy, loudness, valence, tempo, instrumentalness, speechiness))
        # add song data to lists of metrics
            
get_song_metrics(song_uris)


<class 'pandas.core.frame.DataFrame'>
Fetching audio features for 5995 songs.
Song # 0 of 5995
Song # 1 of 5995
Song # 2 of 5995
Song # 3 of 5995
Song # 4 of 5995
Song # 5 of 5995
Song # 6 of 5995
Song # 7 of 5995
Song # 8 of 5995
Song # 9 of 5995
Song # 10 of 5995
Song # 11 of 5995
Song # 12 of 5995
Song # 13 of 5995
Song # 14 of 5995
Song # 15 of 5995
Song # 16 of 5995
Song # 17 of 5995
Song # 18 of 5995
Song # 19 of 5995
Song # 20 of 5995
Song # 21 of 5995
Song # 22 of 5995
Song # 23 of 5995
Song # 24 of 5995
